# This notebook

* Generates .turb setup files for Athena++ to run on Freya
    * Scales the cloud size from previous runs into new trials
* Generates .sh files for running the code

# Setup

In [298]:
%matplotlib widget
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
%load_ext autoreload
%autoreload 2
import pickle


import sys
import os
sys.path.append(os.path.abspath('/freya/ptmp/mpa/wuze/codes'))
from jason import plotting_def, plot_prettier

plotting_def()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Params

## Functions

In [299]:
def _into_dict(start_key, end_key):
    all_keys = list(globals().keys())
    dict_keys = all_keys[all_keys.index(start_key) : all_keys.index(end_key)+1]
    parse_dict = {}
    for key in dict_keys:
        parse_dict[key] = globals()[key]
    return parse_dict

In [300]:
"""Define constants"""

class unit():
    def __init__(self):        
        # length, time, and mass constants
        self.CONST_pc  = 3.086e18
        self.CONST_yr  = 3.154e7
        self.CONST_amu = 1.66053886e-24
        self.CONST_kB  = 1.3806505e-16
        self.unit_length = self.CONST_pc*1e3  # 1 kpc
        self.unit_time   = self.CONST_yr*1e6  # 1 Myr
        self.unit_density = self.CONST_amu    # 1 mp/cm-3
        self.unit_velocity = self.unit_length/self.unit_time
        self.KELVIN = self.unit_velocity*self.unit_velocity*self.CONST_amu/self.CONST_kB
        self.unit_q = (self.unit_density * (self.unit_velocity**3))/self.unit_length
        self.g = 5/3
        
        # avg atomic mass
        Xsol = 1.0
        Zsol = 1.0
        
        X = Xsol * 0.7381
        Z = Zsol * 0.0134
        Y = 1 - X - Z
        
        self.mu  = 1.0/(2.*X+ 3.*(1.-X-Z)/4.+ Z/2.);
        self.mue = 2.0/(1.0+X);
        self.muH = 1.0/X;
        self.mH = 1.0

        # alpha values for different sims
        self.alpha_hyd = 2 ** (1 / 3)  # 1.26
        
        self.alpha_mhd = (2 * 4.2 / 0.77) ** (1 / 3)

u = unit()

In [301]:
def calc_T(P, rho):
    """
    Calculates temeprature from constants
    ----------
    P: gas pressure
    rho: gas density
    """
    T = P/rho * u.KELVIN * u.mu
    return T

def calc_P(T, rho):
    """
    Calculates pressure from constants
    ----------
    T: gas temperature
    rho: gas density
    """
    P = (T / 64**3) * (rho / 64**3) / (u.KELVIN * u.mu)
    return P

def calc_cs(T):
    """
    Calculates sound speed
    ----------
    T: temperature
    mu: avg atomic number of the gas
    """
    # convert to cm
    m_to_cm = 100

    # return np.sqrt(g.g*R*T_hot/M) * m_to_cm/g.unit_velocity
    # return np.sqrt(u.g * u.CONST_kB / (u.mu * u.CONST_amu) * T) * m_to_cm / u.unit_velocity
    return np.sqrt(u.g * u.CONST_kB / (u.mu * u.CONST_amu) * T) / u.unit_velocity

def calc_vturb(mach, P, rho):
    """
    Calculates the turbulent velocity from desired Mach number M
    ----------
    mach: mach number
    P, rho
    """
    T = calc_T(P, rho)
    cs = calc_cs(T)
    return mach * cs
def calc_vturb(mach, T):
    """
    Calculates the turbulent velocity from desired Mach number M
    ----------
    mach: mach number
    T
    """
    cs = calc_cs(T)
    return mach * cs

def calc_mach(v_turb, P, rho):
    """
    Calculates the Mach number
    ----------
    v_turb: turbulence velocity
    P, rho
    """
    T = calc_T(P, rho)
    cs = calc_cs(T)
    print(f'cs = {cs}')
    return v_turb / cs

def calc_dedt_mach(mach, P, rho, L):
    """
    Returns required dedt for a given Mach number, pressure, density, and box size
    """
    # calculate sound speed first
    T = calc_T(P, rho)
    cs_new = calc_cs(T)

    dedt_req = rho * (cs_new**3) * (L**2) * (mach**3) / (u.alpha_hyd**3)
    return dedt_req
def calc_dedt_mach(mach, T, rho, L):
    """
    Returns required dedt for a given Mach number, temperature, and box size
    """
    # calculate sound speed first
    cs_new = calc_cs(T)

    dedt_req = rho * (cs_new**3) * (L**2) * (mach**3) / (u.alpha_hyd**3)
    return dedt_req


def calc_dedt_vturb(v_turb, rho, L):
    """
    Returns required dedt for a given turbulent velocity, density, and box size

    Does NOT calculate sound speed, so no pressure required
    """
    
    dedt_req = rho * v_turb**3 * (L**2) / (u.alpha_hyd**3)
    return dedt_req

def find_ind_l(seq, val):
    seq = list(seq)
    return seq.index(list(filter(lambda x: x>val, seq))[0])

# Load params

In [302]:
trial_old = '240711_0.4_16000'

# turb_hdf_path = f'/freya/ptmp/mpa/wuze/data/cloud_hdfs/mach_{trial_old.split('_')[1]}.athdf'
trial_dep = '240711_0.4_320'; turb_hdf_path = f'/freya/ptmp/mpa/wuze/data/{trial_dep}/turb/Turb.cons.{30:05d}.athdf'
# turb_hdf_path = '../turb/Turb.cons.00100.athdf'

trial_mach = 0  # leave as 0 to be the same

"""SCALING PARAMETERS"""
radius_scale = 2 # the scaling of radius relative to the previous run
rho_scale = 1 # the scaling of density relative to the previous run
clf_number = 0.3  # the Courant, Friedrichs, & Lewy (CFL) Number

# print
print(turb_hdf_path)

/freya/ptmp/mpa/wuze/data/240711_0.4_320/turb/Turb.cons.00030.athdf


In [303]:
# generate new trial
import datetime; date = str(datetime.datetime.now().date()).replace('-', '')[2:]  # six digit date
trial_mach = trial_old.split('_')[1] if trial_mach == 0 else trial_mach

trial_new = f'{date}_{trial_mach}_{float(trial_old.split('_')[-1]) * radius_scale * rho_scale:.0f}'
print(trial_old, trial_new)

import pickle
datapath = f'/freya/ptmp/mpa/wuze/data/{trial_old}'
with open(f'{datapath}/params.pickle', 'rb') as handle:
    run_params_old = pickle.load(handle)

240711_0.4_16000 240716_0.4_32000


# Set params

## Change params

In [304]:
# new params

"""
RUN & BOX PARAMS
"""

## copy and change
run_params_all = run_params_old.copy()
run_params_all['trial'] = trial_new
run_params_all['cloud_radius'] = run_params_old['cloud_radius'] * radius_scale  # multiply radius by scale
run_params_all['rho_hot'] = run_params_old['rho_hot'] * rho_scale  # multiply density by scale
run_params_all['mach'] = float(trial_mach)

## dimensions
run_params_all['grid_dim'] = 256  # scale up the dimensions
run_params_all['grid_mesh'] = 32  # scale up the grid mesh
run_params_all['pfloor'] = 1e-9  # DECREASE the pressure floor

## scale of the box
# run_params_all['box_scale'] = run_params_old['box_scale']  # keep the original box scale
run_params_all['box_scale'] = 50
run_params_all['box_size'] = run_params_all['cloud_radius'] * run_params_all['box_scale']  # get actual box size

run_params_all['turb_cons_hdf_path'] = turb_hdf_path

"""
DERIVED PARAMS
"""
run_params_all['chi'] = run_params_all['T_hot'] / run_params_all['T_cloud']
run_params_all['P_hot'] = calc_P(run_params_all['T_hot'], run_params_all['rho_hot'])  # scaled hot gas pressure
run_params_all['v_turb'] = calc_vturb(run_params_all['mach'], T=run_params_all['T_hot'])  # turbulent velocity from desired mach number
run_params_all['t_eddy'] = run_params_all['box_size']/run_params_all['v_turb']
run_params_all['cs_hot'] = calc_cs(run_params_all['T_hot'])  # sound speed for hot gas

run_params_all['T_mix'] = np.sqrt(run_params_all['T_cloud'] * run_params_all['T_hot'])
run_params_all['T_warm'] = run_params_all['T_mix']  # set to mixed gas temperature, for tracking

# dedt = 0.5 * v_turb^3 = (Mach_num*cs_hot)^3 given the density & box size
run_params_all['dedt'] = calc_dedt_mach(run_params_all['mach'], T=run_params_all['T_hot'], rho=run_params_all['rho_hot'], L=run_params_all['box_size'])

# tcorr ~ t_eddy
run_params_all['t_corr'] = run_params_all['t_eddy']

# dtdrive << t_eddy
run_params_all['dt_drive'] = run_params_all['t_eddy'] / 1e3

# the end time of simulation, multiples of time_start
# reaches >5 t_cc or t_eddysqu
run_params_all['t_cc'] = (run_params_all['chi'] ** (1/2) * run_params_all['cloud_radius'] / run_params_all['v_turb'])
run_params_all['t_maxc'] = np.max([run_params_all['t_eddy'], run_params_all['t_cc']])  # 1 for floored, 1 for start
# the start time of CLOUD simulation; the end time of the TURB simulation
run_params_all['time_start'] = 3 * run_params_all['t_eddy']  # make it 3 x t_eddy
# the end time of CLOUD simulation
run_params_all['time_end'] = 5 * run_params_all['t_maxc'] + run_params_all['time_start']  # make the duration of the cloud runs 5 x t_cc
run_params_all['dt_hdf5'] = run_params_all['t_maxc'] / 10  # the time interval between hdf5 outputs
dt_hdf5_turb = run_params_all['t_eddy'] / 10  # tim interval between hdf5 outputs for TURB
run_params_all['time_cloud'] = run_params_all['time_start']  # introduce cloud when simulation starts

In [305]:
# compare the two
print(f"{'PARAMETER':<20}{'NEW':<30}{'OLD':<30}")
print('-' * 70)
for entry in run_params_all.keys():
    try:  # for floats
        if run_params_all[entry] > 1:  # if a fraction
            print(f"{entry:<20}{run_params_all[entry]:<30.10f}{run_params_old[entry]:<30.10f}")
        else:
            print(f"{entry:<20}{run_params_all[entry]:<30.5e}{run_params_old[entry]:<30.5e}")
    except:  # for strings
        try:
            print(f"{entry:<20}{str(run_params_all[entry]):<30}{str(run_params_old[entry]):<30}")
        except:  # if old does not exist
            try:
                run_params_old[entry]
            except:
                print(f"{entry:<20}{str(run_params_all[entry]):<30}{'NONE EXISTENT':<30}")

PARAMETER           NEW                           OLD                           
----------------------------------------------------------------------
trial               240716_0.4_32000              240711_0.4_16000              
cloud_flag          1.00000e+00                   1.00000e+00                   
T_cloud             800.0000000000                800.0000000000                
cloud_radius        8.00000e-02                   4.00000e-02                   
cloud_pos           [0, 0, 0]                     [0, 0, 0]                     
box_scale           50.0000000000                 50.0000000000                 
box_size            4.0000000000                  2.0000000000                  
grid_dim            256.0000000000                256.0000000000                
grid_vol            2097152.0000000000            2097152.0000000000            
grid_mesh           32.0000000000                 32.0000000000                 
T_floor             800.0000000000    

## Save params

In [306]:
"""Define run parameters"""

class run_params():
    def __init__(self, run_params_all):
        self.trial = run_params_all['trial']
        self.cloud_flag = run_params_all['cloud_flag']
        self.T_cloud = run_params_all['T_cloud']
        self.T_cold = run_params_all['T_cold']
        self.cloud_radius = run_params_all['cloud_radius']
        self.cloud_pos = run_params_all['cloud_pos']
        self.box_size = run_params_all['box_size']
        self.grid_dim = run_params_all['grid_dim']
        self.grid_vol = run_params_all['grid_vol']
        self.grid_mesh = run_params_all['grid_mesh']
        self.pfloor = run_params_all['pfloor']
        self.T_floor = run_params_all['T_floor']
        self.T_ceil = run_params_all['T_ceil']
        self.cooling_flag = run_params_all['cooling_flag']
        self.T_hot = run_params_all['T_hot']
        self.rho_hot = run_params_all['rho_hot']
        self.mach = run_params_all['mach']
        self.T_cut = run_params_all['T_cut']
        self.chi = run_params_all['chi']
        self.P_hot = run_params_all['P_hot']
        self.v_turb = run_params_all['v_turb']
        self.t_eddy = run_params_all['t_eddy']
        self.cs_hot = run_params_all['cs_hot']
        self.T_mix = run_params_all['T_mix']
        self.T_warm = run_params_all['T_warm']
        self.dedt = run_params_all['dedt']
        self.t_corr = run_params_all['t_corr']
        self.dt_drive = run_params_all['dt_drive']
        self.t_cc = run_params_all['t_cc']
        self.t_maxc = run_params_all['t_maxc']
        self.time_start = run_params_all['time_start']
        self.time_end = run_params_all['time_end']
        self.dt_hdf5 = run_params_all['dt_hdf5']
        self.time_cloud = run_params_all['time_cloud']

rp = run_params(run_params_all)
rp.trial

'240716_0.4_32000'

In [307]:
import os

# make datapath
datapath = f'/freya/ptmp/mpa/wuze/data/{rp.trial}'

if not os.path.exists(datapath):
    os.makedirs(datapath)
    os.makedirs(f'{datapath}/cloud')
    os.makedirs(f'{datapath}/turb')
    print(f'Made path {datapath}')

Made path /freya/ptmp/mpa/wuze/data/240716_0.4_32000


In [308]:
# save as pickles
# the dictionary saved here is separate from the rp object
import pickle

with open(f'{datapath}/params.pickle', 'wb') as handle:
    pickle.dump(run_params_all, handle, protocol=pickle.HIGHEST_PROTOCOL)

run_params_all

{'trial': '240716_0.4_32000',
 'cloud_flag': True,
 'T_cloud': 800.0,
 'cloud_radius': 0.08,
 'cloud_pos': [0, 0, 0],
 'box_scale': 50,
 'box_size': 4.0,
 'grid_dim': 256,
 'grid_vol': 2097152,
 'grid_mesh': 32,
 'T_floor': 800.0,
 'T_cold': 1600.0,
 'T_ceil': 100000000.0,
 'cooling_flag': True,
 'T_hot': 4000000.0,
 'rho_hot': 0.1,
 'mach': 0.4,
 'turb_cons_hdf_path': '/freya/ptmp/mpa/wuze/data/240711_0.4_320/turb/Turb.cons.00030.athdf',
 'T_cut': 1000000.0,
 'chi': 5000.0,
 'P_hot': 8.438675409534824e-14,
 'v_turb': 0.12435447811137146,
 't_eddy': 32.16611143201143,
 'cs_hot': 0.31088619527842865,
 'T_mix': 56568.5424949238,
 'T_warm': 56568.5424949238,
 'dedt': 0.0015384177235386331,
 't_corr': 32.16611143201143,
 'dt_drive': 0.032166111432011424,
 't_cc': 45.48975103595482,
 't_maxc': 45.48975103595482,
 'time_start': 96.49833429603427,
 'time_end': 323.94708947580835,
 'dt_hdf5': 4.548975103595482,
 'time_cloud': 96.49833429603427,
 'pfloor': 1e-09}

# Gen .turb

## Readme

In [309]:
"""
Generate readme
"""

def infgen_readme(datapath, readme_msg="This run"):
    f = open(f"{datapath}/readme", "w")
    
    # generate text
    readme_text =\
    f"""
{readme_msg}

{'-' * 50}
This run is auto-generated by the trial_gen.ipynb

This run uses:
    """
    
    # generate variables
    readme_vars = f'\n{'-' * 20}\n'
    for key, val in run_params_all.items():
        if isinstance(val, float):
            app = f'{key} = {val:.4f}\n'
        else:
            app = f'{key} = {val}\n'
        readme_vars += app
    
    f.write(readme_text)
    f.write('-' * 50)
    f.write(readme_vars)
    f.close()

## athinput_init.turb

In [310]:
"""
Generate TURB .turb file
"""
def infgen_init_turb(datapath):
    f = open(f"{datapath}/turb/athinput_init.turb", "w")
    
    # generate text
    turb_config =\
    f""" 
<comment>
problem   = Adding turbulence
configure = --prob=turb_v2 -fft 

<job>
problem_id = Turb # problem ID: basename of output filenames


#~----------OUTPUTS-----------~#
<output1>
file_type  = hst        # history data dump
dt         = {dt_hdf5_turb / 100 :<20.10f}# time increment between outputs

<output2>
file_type = hdf5        # HDF5 data dump
variable  = prim        # variables to be output
dt        = {dt_hdf5_turb:<20.10f}# time increment between outputs

<output3>
file_type = rst         # restart file dump                       
dt        = {dt_hdf5_turb * 10 :<20.10f}# time increment between outputs

<output4>
file_type = hdf5        # HDF5 data dump
variable  = cons        # variables to be output
dt        = {dt_hdf5_turb:<20.10f}# time increment between outputs
id        = cons

#~----------SETUP-----------~#
<time>
cfl_number  = {clf_number:<20}# The Courant, Friedrichs, & Lewy (CFL) Number
nlim        = -1        # cycle limit
# time limit of second trial should be larger
tlim        = {run_params_all['time_start']:<20.10f}# time limit for the turb run, where the cloud run starts
integrator  = rk2       # time integration algorithm
xorder      = 2         # order of spatial reconstruction
ncycle_out  = 1         # interval for stdout summary info

<mesh>
nx1    = {run_params_all['grid_dim']}             # number of zones in x1-direction
x1min  = {-run_params_all['box_size']/2}           # minimum value of x1
x1max  = {run_params_all['box_size']/2}            # maximum value of x1
ix1_bc = periodic       # inner-x1 boundary condition
ox1_bc = periodic       # outer-x1 boundary condition

nx2    = {run_params_all['grid_dim']}             # number of zones in x2-direction
x2min  = {-run_params_all['box_size']/2}           # minimum value of x2
x2max  = {run_params_all['box_size']/2}            # maximum value of x2
ix2_bc = periodic       # inner-x2 boundary condition
ox2_bc = periodic       # outer-x2 boundary condition

nx3    = {run_params_all['grid_dim']}             # number of zones in x3-direction
x3min  = {-run_params_all['box_size']/2}           # minimum value of x3
x3max  = {run_params_all['box_size']/2}            # maximum value of x3
ix3_bc = periodic       # inner-x3 boundary condition
ox3_bc = periodic       # outer-x3 boundary condition

refinement  = none      # type of mesh refinement to use

<meshblock>
nx1 = {run_params_all['grid_mesh']}                # block size in x1-direction
nx2 = {run_params_all['grid_mesh']}                # block size in x2-direction
nx3 = {run_params_all['grid_mesh']}                # block size in x3-direction

<hydro>
gamma      = 1.6666666666666667  # gamma = C_p/C_v
pfloor     = {run_params_all['pfloor']:<20.10f}  # pressure floor

#~----------TURBULENCE PARAMS-----------~#
<turbulence>
dedt       = {run_params_all['dedt']:<20.15f}# Energy injection rate (for driven) or Total energy (for decaying)
nlow       = 0                   # cut-off wavenumber at low-k
nhigh      = 2                   # cut-off wavenumber at high-k
expo       = 2.0                 # power-law exponent
tcorr      = {run_params_all['t_corr']:<20.15f}# correlation time for OU process (both impulsive and continuous)
dtdrive    = {run_params_all['dt_drive']:<20.15f}# time interval between perturbation (impulsive)
f_shear    = 0.3                 # the ratio of the shear component
rseed      = 1                   # if non-negative, seed will be set by hand (slow PS generation)
# dedt should be calibrated by        dedt = 0.5 * v_turb^3 = (Mach_num*cs_hot)^3
# tcorr ~ t_eddy
# dtdrive << t_eddy

<problem>
turb_flag    = 2                 # 1 for decaying, 2 (impulsive) or 3 (continuous) for driven turbulence
rescale_flag = 1                 # 1 for cloud runs


#~----------HEATING & COOLING-----------~#
# User-defined variables:

heating = 0.001                  # constant volume heating rate

cs_hot = {run_params_all['cs_hot']:<20.10f}# hot gas sound speed for T~4e6K

# turn on cooling for cloud runs
cooling_flag = 0                 # set to 1 to turn on cooling, 0 to turn it off
global_cooling_flag = 0          # set to 1 to turn on uniform global cooling, 0 to turn it off
# turn on cloud
restart_cloud_flag   = 0                 # set to 1 to add the cloud on restart, 0 to not 
hdf_cloud_flag       = 0                 # set to 1 to add the cloud after reading HDF5 file, 0 to not 

amb_rho      = {run_params_all['rho_hot']}               # density of the ambient medium, in code units


#~----------CLOUD PROPERTIES-----------~#
cloud_radius = {run_params_all['cloud_radius']:<20.10f}# radius of the cloud, in code units
# this should be the same as simulation start
start_time   = {0:<20.10f}# time of insertion of cloud, in code units
cloud_time   = {run_params_all['time_cloud']:<20.10f}# time of insertion of cloud, in code units
# temperature ratio T_init / T_cloud
# TO SET COLD GAS TEMPERATURE, CHANGE HERE
cloud_chi    = {run_params_all['chi']}               # density contrast of the cloud, rho_cloud/amb_rho

cloud_pos_x  = {run_params_all['cloud_pos'][0]}               # cloud center position x-coordinate
cloud_pos_y  = {run_params_all['cloud_pos'][1]}               # cloud center position y-coordinate
cloud_pos_z  = {run_params_all['cloud_pos'][2]}               # cloud center position z-coordinate

#~-----------TEMPERATURE----------~#
T_floor      = {run_params_all['T_floor']:<10.0f}# floor temperature in the simulation
T_ceil       = {run_params_all['T_ceil']:<10.0f}# ceiling temperature in the simulation

# medium mass is integrated from 2 x temp
T_hot_req    = {run_params_all['T_hot']:<10.0f}# hot medium temperature required, reset to this on restart, if cloud_flag is 1
T_hot        = {run_params_all['T_hot']:<10.0f}# initial hot medium temperature (box heats up due to turbulence)
T_cold       = {run_params_all['T_cold']:<10.0f}# cold medium temperature, ONLY used in cold gas mass calculation
T_cut_mul    = 0.5               # T_cut = T_hot_req * T_cut_mul, gas higher than T_cut is not cooled
# infinite cooling time for T > T_cut, want hot gas to stay hot for the medium
T_cut        = {run_params_all['T_cut']:<10.0f}# gas higher than T_cut is not cooled
T_warm       = {run_params_all['T_warm']:<10.2f}# warm medium temperature, ONLY used in warm gas mass calculation

Xsol         = 1.0               # Change H-abundance src/utils/code_units
Zsol         = 1.0               # Change metallicity src/utils/code_units

B_x          = 0.0               # initial magnetic field in x-direction
B_y          = 0.0               # initial magnetic field in y-direction
B_z          = 0.0               # initial magnetic field in z-direction

# only read if hdf_cloud_flag = 1
cons_input_filename = /dev/null  # name of HDF5 file containing initial conditions
dataset_cons   = cons       # name of dataset containing conserved values
index_dens     = 0          # index of density in conserved dataset
index_etot     = 1          # index of energy in conserved dataset (for nonbarotropic EOS)
index_mom1     = 2          # index of x1-momentum in conserved dataset
index_mom2     = 3          # index of x2-momentum in conserved dataset
index_mom3     = 4          # index of x3-momentum in conserved dataset

# only used if -b flag is on for compiling
b1_input_filename = /dev/null  # name of HDF5 file containing initial conditions
b2_input_filename = /dev/null  # name of HDF5 file containing initial conditions
b3_input_filename = /dev/null  # name of HDF5 file containing initial conditions
    """
    
    # generate variables
    f.write(turb_config)
    f.write('-' * 50)
    f.close()

## athinput_cloud.turb

In [311]:
"""
Generate CLOUD .turb file
"""
def infgen_cloud_turb(datapath):
    f = open(f"{datapath}/cloud/athinput_cloud.turb", "w")
    
    # generate text
    cloud_config =\
    f"""
<comment>
problem   = Adding cloud
configure = --prob=turb_v2 -fft 

<job>
problem_id = Turb # problem ID: basename of output filenames


#~----------OUTPUTS-----------~#
<output1>
file_type  = hst        # history data dump
dt         = {run_params_all['dt_hdf5'] / 100 :<20.10f}# time increment between outputs

<output2>
file_type = hdf5        # HDF5 data dump
variable  = prim        # variables to be output
dt        = {run_params_all['dt_hdf5']:<20.10f}# time increment between outputs

<output3>
file_type = rst         # restart file dump                       
dt        = {run_params_all['dt_hdf5'] * 10 :<20.10f}# time increment between outputs

<output4>
file_type = hdf5        # HDF5 data dump
variable  = cons        # variables to be output
dt        = {run_params_all['dt_hdf5']:<20.10f}# time increment between outputs
id        = cons

#~----------SETUP-----------~#
<time>
cfl_number  = {clf_number:<20}# The Courant, Friedrichs, & Lewy (CFL) Number
nlim        = -1        # cycle limit
# time limit of second trial should be larger
tlim        = {run_params_all['time_end']:<20.10f}# time limit
integrator  = rk2       # time integration algorithm
xorder      = 2         # order of spatial reconstruction
ncycle_out  = 1         # interval for stdout summary info

<mesh>
nx1    = {run_params_all['grid_dim']}             # number of zones in x1-direction
x1min  = {-run_params_all['box_size']/2}           # minimum value of x1
x1max  = {run_params_all['box_size']/2}            # maximum value of x1
ix1_bc = periodic       # inner-x1 boundary condition
ox1_bc = periodic       # outer-x1 boundary condition

nx2    = {run_params_all['grid_dim']}             # number of zones in x2-direction
x2min  = {-run_params_all['box_size']/2}           # minimum value of x2
x2max  = {run_params_all['box_size']/2}            # maximum value of x2
ix2_bc = periodic       # inner-x2 boundary condition
ox2_bc = periodic       # outer-x2 boundary condition

nx3    = {run_params_all['grid_dim']}             # number of zones in x3-direction
x3min  = {-run_params_all['box_size']/2}           # minimum value of x3
x3max  = {run_params_all['box_size']/2}            # maximum value of x3
ix3_bc = periodic       # inner-x3 boundary condition
ox3_bc = periodic       # outer-x3 boundary condition

refinement  = none      # type of mesh refinement to use

<meshblock>
nx1 = {run_params_all['grid_mesh']}                # block size in x1-direction
nx2 = {run_params_all['grid_mesh']}                # block size in x2-direction
nx3 = {run_params_all['grid_mesh']}                # block size in x3-direction

<hydro>
gamma      = 1.6666666666666667  # gamma = C_p/C_v
pfloor     = {run_params_all['pfloor']:<20.10f}  # pressure floor

#~----------TURBULENCE PARAMS-----------~#
<turbulence>
dedt       = {run_params_all['dedt']:<20.15f}# Energy injection rate (for driven) or Total energy (for decaying)
nlow       = 0                   # cut-off wavenumber at low-k
nhigh      = 2                   # cut-off wavenumber at high-k
expo       = 2.0                 # power-law exponent
tcorr      = {run_params_all['t_corr']:<20.15f}# correlation time for OU process (both impulsive and continuous)
dtdrive    = {run_params_all['dt_drive']:<20.15f}# time interval between perturbation (impulsive)
f_shear    = 0.3                 # the ratio of the shear component
rseed      = 1                   # if non-negative, seed will be set by hand (slow PS generation)
# dedt should be calibrated by        dedt = 0.5 * v_turb^3 = (Mach_num*cs_hot)^3
# tcorr ~ t_eddy
# dtdrive << t_eddy

<problem>
turb_flag    = 2                 # 1 for decaying, 2 (impulsive) or 3 (continuous) for driven turbulence
rescale_flag = 1                 # 1 for cloud runs


#~----------HEATING & COOLING-----------~#
# User-defined variables:

heating = 0.001                  # constant volume heating rate

cs_hot = {run_params_all['cs_hot']:<20.10f}# hot gas sound speed for T~4e6K

# turn on cooling for cloud runs
cooling_flag = {1 if run_params_all['cooling_flag'] else 0}                 # set to 1 to turn on cooling, 0 to turn it off
global_cooling_flag = 0          # set to 1 to turn on uniform global cooling, 0 to turn it off
# turn on cloud
restart_cloud_flag   = 0                 # set to 1 to add the cloud on restart, 0 to not 
hdf_cloud_flag       = {1 if run_params_all['cloud_flag'] else 0}                 # set to 1 to add the cloud after reading HDF5 file, 0 to not 

amb_rho      = {run_params_all['rho_hot']}               # density of the ambient medium, in code units


#~----------CLOUD PROPERTIES-----------~#
cloud_radius = {run_params_all['cloud_radius']:<20.10f}# radius of the cloud, in code units
# this should be the same as simulation start
start_time   = {run_params_all['time_start']:<20.10f}# time of insertion of cloud, in code units
cloud_time   = {run_params_all['time_cloud']:<20.10f}# time of insertion of cloud, in code units
# temperature ratio T_init / T_cloud
# TO SET COLD GAS TEMPERATURE, CHANGE HERE
cloud_chi    = {run_params_all['chi']}               # density contrast of the cloud, rho_cloud/amb_rho

cloud_pos_x  = {run_params_all['cloud_pos'][0]}               # cloud center position x-coordinate
cloud_pos_y  = {run_params_all['cloud_pos'][1]}               # cloud center position y-coordinate
cloud_pos_z  = {run_params_all['cloud_pos'][2]}               # cloud center position z-coordinate

#~-----------TEMPERATURE----------~#
T_floor      = {run_params_all['T_floor']:<10.0f}# floor temperature in the simulation
T_ceil       = {run_params_all['T_ceil']:<10.0f}# ceiling temperature in the simulation

# medium mass is integrated from 2 x temp
T_hot_req    = {run_params_all['T_hot']:<10.0f}# hot medium temperature required, reset to this on restart, if cloud_flag is 1
T_hot        = {run_params_all['T_hot']:<10.0f}# initial hot medium temperature (box heats up due to turbulence)
T_cold       = {run_params_all['T_cold']:<10.0f}# cold medium temperature, ONLY used in cold gas mass calculation
T_cut_mul    = 0.5               # T_cut = T_hot_req * T_cut_mul, gas higher than T_cut is not cooled
# infinite cooling time for T > T_cut, want hot gas to stay hot for the medium
T_cut        = {run_params_all['T_cut']:<10.0f}# gas higher than T_cut is not cooled
T_warm       = {run_params_all['T_warm']:<10.2f}# warm medium temperature, ONLY used in warm gas mass calculation

Xsol         = 1.0               # Change H-abundance src/utils/code_units
Zsol         = 1.0               # Change metallicity src/utils/code_units

B_x          = 0.0               # initial magnetic field in x-direction
B_y          = 0.0               # initial magnetic field in y-direction
B_z          = 0.0               # initial magnetic field in z-direction

# only read if hdf_cloud_flag = 1
cons_input_filename = {run_params_all['turb_cons_hdf_path']}  # name of HDF5 file containing initial conditions
dataset_cons   = cons       # name of dataset containing conserved values
index_dens     = 0          # index of density in conserved dataset
index_etot     = 1          # index of energy in conserved dataset (for nonbarotropic EOS)
index_mom1     = 2          # index of x1-momentum in conserved dataset
index_mom2     = 3          # index of x2-momentum in conserved dataset
index_mom3     = 4          # index of x3-momentum in conserved dataset

# only used if -b flag is on for compiling
b1_input_filename = /dev/null  # name of HDF5 file containing initial conditions
b2_input_filename = /dev/null  # name of HDF5 file containing initial conditions
b3_input_filename = /dev/null  # name of HDF5 file containing initial conditions
    """
    
    # generate variables
    f.write(cloud_config)
    f.write('-' * 50)
    f.close()

## Write files

In [312]:
# readme
readme_msg = """

THIS IS ONE OF THE NEW RUNS,
check notebook on 24/7/12!

These runs use:
256 cell
1/50 cloud radius
1e-9 pfloor
0.3 cfl

"""
infgen_readme(datapath=datapath, readme_msg=readme_msg)

In [313]:
# init / turb
infgen_init_turb(datapath=datapath)

In [314]:
# cloud
infgen_cloud_turb(datapath=datapath)

In [315]:
import datetime
print(datetime.datetime.now())
print('\n' * 10)

2024-07-16 19:48:45.525173













In [316]:
pass

# Gen .sh

In [317]:
"""
Get trial params
"""
import datetime
date = str(datetime.datetime.now().date()).replace('-', '')[2:]  # six digit date
run_dp = trial_new[7:]  # x_y

## Turb.sh

In [318]:
"""
Machine params
"""
executable = 'athena_turb'
server = '24h' #test
if server == '24h':
    comp_time = '3:55:00'#'23:55:00'
    comp_time_save = '3:50:00'#'23:50:00'
elif server == 'test':
    comp_time = '00:20:00'
    comp_time_save = '00:19:00'

print(date, run_dp, executable, server, comp_time, comp_time_save)

240716 0.4_32000 athena_turb 24h 3:55:00 3:50:00


In [319]:
sh_content = f"""#!/usr/bin/env bash
# source ~/.bashrc

#SBATCH -J {date[2:]}{trial_new[6:]}_turb
#SBATCH -o ./out/{date}/{run_dp}_turb."%j".out
#SBATCH -e ./out/{date}/{run_dp}_turb."%j".err
#SBATCH --mail-user wuz@mpa-garching.mpg.de
#-- wuz@mpa-garching.mpg.de
#SBATCH --partition=p.24h
#-- p.24h, p.test, p.gpu & p.gpu.ampere
#SBATCH --mail-type=ALL
#SBATCH --nodes=16
#-- 2 at most?
#SBATCH --ntasks-per-node=32
#-- 40 at most
#SBATCH --time={comp_time}
#-- in format hh:mm:ss

set -e
SECONDS=0

module purge
module load intel/19.1.2
module load impi/2019.8
module load fftw-mpi/3.3.8
module load hdf5-mpi/1.8.21
module load ffmpeg/4.4
module list

echo {trial_new}
cd $DP/{trial_new}/turb

srun ../../{executable} -i athinput_init.turb -t {comp_time_save}
#-- !!TIME_LIMIT_RST!! should be ten minutes before the end, to generate a restart file

echo "Elapsed: $(($SECONDS / 3600))hrs $((($SECONDS / 60) % 60))min $(($SECONDS % 60))sec"

echo "Boom!"

cd ~
"""

# write the file
file_name = f"{datapath}/js_turb_24h.sh" if server == '24h' else f"{datapath}/js_turb.sh"
with open(file_name, 'w') as file:
    file.write(sh_content)

# print the squeue command
squeue_cmd = f"""
sbatch {datapath}/js_turb_24h.sh
"""
print(squeue_cmd)


sbatch /freya/ptmp/mpa/wuze/data/240716_0.4_32000/js_turb_24h.sh



## Cloud.sh

In [320]:
"""
Machine params
"""
executable = 'athena_stop'#'athena_turb'
server = '24h' #test
if server == '24h':
    comp_time = '5:55:00'#'23:55:00'
    comp_time_save = '5:50:00'#'23:50:00'
elif server == 'test':
    comp_time = '00:20:00'
    comp_time_save = '00:19:00'

print(date, run_dp, executable, server, comp_time, comp_time_save)

240716 0.4_32000 athena_stop 24h 5:55:00 5:50:00


In [321]:
sh_content = f"""#!/usr/bin/env bash
# source ~/.bashrc

#SBATCH -J {date[2:]}{trial_new[6:]}_cloud
#SBATCH -o ./out/{date}/{run_dp}_cloud."%j".out
#SBATCH -e ./out/{date}/{run_dp}_cloud."%j".err
#SBATCH --mail-user wuz@mpa-garching.mpg.de
#-- wuz@mpa-garching.mpg.de
#SBATCH --partition=p.24h
#-- p.24h, p.test, p.gpu & p.gpu.ampere
#SBATCH --mail-type=ALL
#SBATCH --nodes=16
#-- 2 at most?
#SBATCH --ntasks-per-node=32
#-- 40 at most
#SBATCH --time={comp_time}
#-- in format hh:mm:ss

set -e
SECONDS=0

module purge
module load intel/19.1.2
module load impi/2019.8
module load fftw-mpi/3.3.8
module load hdf5-mpi/1.8.21
module load ffmpeg/4.4
module list

echo {trial_new}
cd $DP/{trial_new}/cloud

srun ../../{executable} -i athinput_cloud.turb -t {comp_time_save}
#-- !!TIME_LIMIT_RST!! should be ten minutes before the end, to generate a restart file

echo "Elapsed: $(($SECONDS / 3600))hrs $((($SECONDS / 60) % 60))min $(($SECONDS % 60))sec"

echo "Boom!"

cd ~
"""

# write the file
file_name = f"{datapath}/js_cloud_24h.sh" if server == '24h' else f"{datapath}/js_cloud.sh"
with open(file_name, 'w') as file:
    file.write(sh_content)

# print the squeue command
squeue_cmd = f"""
sbatch {datapath}/js_cloud_24h.sh
"""
print(squeue_cmd)

# print the squeue command
squeue_cmd = f"""
sbatch --dependency=afterok:XXX {datapath}/js_cloud_24h.sh
"""
print(squeue_cmd)


sbatch /freya/ptmp/mpa/wuze/data/240716_0.4_32000/js_cloud_24h.sh


sbatch --dependency=afterok:XXX /freya/ptmp/mpa/wuze/data/240716_0.4_32000/js_cloud_24h.sh



In [322]:
!mkdir -p /freya/ptmp/mpa/wuze/out/{date}

In [323]:
print('\n' * 50)

# Time to 24h

In [324]:
# trials = '240620_0.3_160 240620_0.5_160 240620_0.5_16 240620_0.8_320 240620_0.6_320 240620_0.3_800 240620_0.5_800 240620_0.8_1600 240620_0.6_1600'.split(' ')

# import os
# for trial in trials:
#     datapath = f'/freya/ptmp/mpa/wuze/data/{trial}'
#     with open(f'{datapath}/js_cloud.sh', 'r') as file:
#         file_content = file.read()
    
#     # remove old hst file
#     try:
#         os.remove(f'{datapath}/cloud/Turb.hst')
#         print(f'Delted file in {datapath}')
#     except OSError:
#         pass
#     """Replace with 24h and new output"""
#     new_content = file_content.replace('240620/', '240621/')
#     new_content = new_content.replace('04:00:00', '23:00:00')
#     new_content = new_content.replace('03:55:00', '22:55:00')

#     # write as new file
#     with open(f'{datapath}/js_cloud_24h.sh', 'w') as new_file:
#         new_file.write(new_content)

#     squeue_cmd = f"""
#     sbatch {datapath}/js_cloud_24h.sh
#     """
#     print(squeue_cmd)